In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, \
MaxPooling2D, Conv2D, Lambda, Cropping2D, Convolution2D,\
AveragePooling2D
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint


from sklearn.utils import shuffle

import tensorflow as tf
#tf.python.control_flow_ops = tf

epochs = 50
batch_size = 128
image_columns = 32
image_rows = 16
image_channels = 1
side_shift = 0.3



Using TensorFlow backend.


In [6]:
size=(40,80)

def preprocessing(im):
    return cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2YUV),(size[1],size[0]))

In [7]:
def load(csv, correction,folder):
    df=pd.read_csv(folder+csv, skiprows=1,\
            names=['center','left','right','steering','trottle','brake','speed'])
    
    df['center']=df.center.apply(lambda x:x.split('/')[-1])
    df['left']=df.left.apply(lambda x:x.split('/')[-1])
    df['right']=df.right.apply(lambda x:x.split('/')[-1])

    images=np.empty((len(df)*6,size[0],size[1],3), dtype=np.uint8)
    target=np.empty((len(df))*6, dtype=np.float)

    #center
    for i,fname in enumerate(df['center'].values):
        images[i]=preprocessing(cv2.imread(folder+'IMG/'+fname.strip()))

    #left
    for i,fname in enumerate(df['left'].values):
        images[i+len(df)]=preprocessing(cv2.imread(folder+'IMG/'+fname.strip()))

    #right
    for i,fname in enumerate(df['right'].values):
        images[i+2*len(df)]=preprocessing(cv2.imread(folder+'IMG/'+fname.strip()))

    target[0:len(df)]=df.steering.values
    target[len(df):2*len(df)]=df.steering.values+correction
    target[2*len(df):3*len(df)]=df.steering.values-correction

    target[3*len(df):6*len(df)]=-target[0:3*len(df)]

    for i in range(len(df)*3):
        images[i+3*len(df)]=np.fliplr(images[i])
        
    return images, target

In [11]:
if not os.path.exists('X.npy'):

    data=[]
    for path in ('/notebooks/udacity/new_training/map1_backward/',
                 '/notebooks/udacity/new_training/map1_forward/',
                 '/notebooks/udacity/new_training/map1_recovery_backward/',
                 '/notebooks/udacity/new_training/map1_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_forward/',
                 '/notebooks/udacity/new_training/map2_backward/',
                 '/notebooks/udacity/new_training/map2_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_recovery_backward/'
                ):
        data.append(load('driving_log.csv',0.25,path))
    
    X=np.concatenate([a[0] for a in data])
    y=np.concatenate([a[1] for a in data])
    np.save('X.npy',X)
    np.save('y.npy',y)
    del (data)
else:
    X=np.load('X.npy')
    y=np.load('y.npy')

In [12]:
print (X.shape, y.shape)

(231162, 40, 80, 3) (231162,)


In [13]:
model=Sequential()

model.add(Cropping2D(cropping=((12,4), (0,0)), \
                     input_shape=(40,80,3)))

model.add(Lambda(lambda x: (x / 255.0) - 0.5))

model.add(AveragePooling2D(pool_size=(4,4)))



model.add(Convolution2D(32,3, 3,
                 activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))

In [14]:
model.compile(loss='mse',\
              optimizer='adam', metrics=['mean_squared_error'])

In [53]:
def preprocessing(im):
    return cv2.cvtColor(im, cv2.COLOR_BGR2YUV)

In [15]:
from keras.preprocessing.image import ImageDataGenerator

generator=ImageDataGenerator(rotation_range=2, \
                             width_shift_range=0.03,\
                            height_shift_range=0.03,
                            fill_mode='nearest')

In [16]:
from sklearn.cross_validation import train_test_split

train, test= train_test_split(range(len(X)), test_size=0.15)

/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [17]:
checkpoint = ModelCheckpoint("model_all.h5", monitor='val_mean_squared_error', verbose=1,
                              save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=2,
                            verbose=1, mode='min')

history = History()

tb=TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)


In [18]:
gen=generator.flow(X[train],y[train],batch_size=128,shuffle=True)

In [19]:
X.shape, y.shape

((231162, 40, 80, 3), (231162,))

In [20]:
with open("model_all.json", "w") as json_file:
    json_file.write(model.to_json())

In [ ]:
model.fit_generator(gen, \
                    samples_per_epoch=len(train),\
                    nb_epoch=100,\
                    verbose=1,\
                    validation_data=(X[test],y[test]),\
                    callbacks=[checkpoint, early_stop]\
                   )

Epoch 1/100
196487/196487 [==============================] - 109s - loss: 0.0847 - mean_squared_error: 0.0847 - val_loss: 0.0703 - val_mean_squared_error: 0.0703
Epoch 2/100
 51072/196487 [======>.......................] - ETA: 80s - loss: 0.0770 - mean_squared_error: 0.0770

In [59]:
with open("model_nvidia.json", "w") as json_file:
    json_file.write(model.to_json())